In [1]:
import h5py # for working with HDF5 (Hierarchical Data Format) files
import numpy as np
import sklearn.svm
import matplotlib.pyplot as plt
import pandas as pd
import os
from V3.load_calcium_video import load_video_data, load_one_video
from V3.align_behavior_to_calcium import align_files_old_labels, align_files_new_labels
from V3.class_balance import check_class_imbalance_old, check_class_imbalance_new 
from V3.preprocessing_model import model_preprocessing

2023-05-11 14:05:35.100341: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-11 14:05:35.138133: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
comment = "k-fold-cross validation, multiple videos, new labels"
experiment_ID = '3.4'
data_file = 'Animal3learnday8, Animal3Learnday9, Animal3Learnday10'
experiment_name = str(data_file)+"_"+str(experiment_ID)
train_test_split_strategy = "k-fold"
name = 'BPSVM'
model_version = str(name)+'_1'

%store experiment_ID

Stored 'experiment_ID' (str)


In [3]:
# Which labels am I using? Old or New?
labels_type = 'new' # or 'old'

In [4]:
# Shuffle data? This to find chance performance
shuffled_labels = False

In [5]:
# Am I using one or multiple videos for training?
multiple_videos = False 

In [6]:
# Define the name of the output directory
output_dir = "output-svm"

# Check if the output directory already exists
if not os.path.exists(output_dir):
    # Create the output directory
    os.mkdir(output_dir)

    # Create the balance, accuracy, loss, and cm directories inside the output directory
    os.mkdir(os.path.join(output_dir, "balance"))
    os.mkdir(os.path.join(output_dir, "accuracy"))
    os.mkdir(os.path.join(output_dir, "loss"))
    os.mkdir(os.path.join(output_dir, "cm"))
    os.mkdir(os.path.join(output_dir, "architecture"))
    os.mkdir(os.path.join(output_dir, "pickles"))
else:
    print(f"The directory {output_dir} already exists.")

The directory output-svm already exists.


### Loading Calcium Videos

In [7]:
# # paths to videos
# video_paths = ["/home/dmc/Desktop/kostas/direct-Behavior-prediction-from-miniscope-calcium-imaging-using-convolutional-neural-networks/data/20211025_184906_animal3learnday8.nwb", 
#                "/home/dmc/Desktop/kostas/direct-Behavior-prediction-from-miniscope-calcium-imaging-using-convolutional-neural-networks/data/20211026_142935_animal3learnday9.nwb", 
#                "/home/dmc/Desktop/kostas/direct-Behavior-prediction-from-miniscope-calcium-imaging-using-convolutional-neural-networks/data/20211027_165052_animal3learnday10.nwb"]

video_path = ["/home/dmc/Desktop/kostas/direct-Behavior-prediction-from-miniscope-calcium-imaging-using-convolutional-neural-networks/data/20211028_181307_animal3learnday11.nwb"]

In [8]:
# load the CSV file with the FOV information
# fov_info = pd.read_csv('/home/dmc/Desktop/kostas/direct-Behavior-prediction-from-miniscope-calcium-imaging-using-convolutional-neural-networks/src/V3/aligned_videos_animal3.csv')

In [9]:
video_name_list = []
video_data_list = []

if multiple_videos == True:
    images = load_video_data(video_paths, fov_info, video_name_list, video_data_list)
else:
    # for one video
    images = load_one_video(video_path, video_name_list, video_data_list)

20211028_181307_animal3learnday11.nwb recording_20211028_181307-PP-BP-MC
(24589, 393, 444)


In [10]:
images[0].shape

(393, 444)

In [11]:
# Check if the video has been loaded correctly
images[:1]

array([[[175, 181, 182, ..., 181, 181, 180],
        [170, 174, 173, ..., 177, 176, 179],
        [175, 175, 168, ..., 180, 176, 178],
        ...,
        [180, 182, 175, ..., 170, 173, 167],
        [177, 188, 181, ..., 177, 179, 180],
        [184, 184, 183, ..., 188, 182, 174]]], dtype=int16)

In [12]:
# Determine the size of the calcium video dataset
num_of_frames = images.shape[0]
img_height = images.shape[1]
img_width = images.shape[2]
print("The number of video frames is ", num_of_frames, " and the frame dimensions (height x width) are: ", img_height, "X", img_width)

The number of video frames is  24589  and the frame dimensions (height x width) are:  393 X 444


### Loading Calcium Traces

In [13]:
path = "/home/dmc/Desktop/kostas/direct-Behavior-prediction-from-miniscope-calcium-imaging-using-convolutional-neural-networks/animal3.h5"

In [14]:
# manually change from which videos you want to extract calcium traces
list_of_traces = []

if multiple_videos == True:
    with (h5py.File(path, 'r')) as f:
        traces_8 = np.array(f['traces/animal3learnday8/deconvolved'])
        list_of_traces.append(traces_8)
        traces_9 = np.array(f['traces/animal3learnday9/deconvolved'])
        list_of_traces.append(traces_9)
        traces_10 = np.array(f['traces/animal3learnday10/deconvolved'])
        list_of_traces.append(traces_10)
else:
    with (h5py.File(path, 'r')) as f:
        traces = np.array(f['traces/animal3learnday11/deconvolved'])
        # list_of_traces.append(traces_11)

In [15]:
# traces = np.concatenate(list_of_traces, axis=1)
# print("Concatenated traces shape:", traces.shape)

### Load Labels

In [16]:
if multiple_videos == True:
    num_of_videos = 3
else:
    num_of_videos = 1

In [17]:


if multiple_videos == True:
    bonsai_paths = ["/home/dmc/Desktop/kostas/direct-Behavior-prediction-from-miniscope-calcium-imaging-using-convolutional-neural-networks/data/tmaze_2021-10-25T18_48_49.csv", 
                   "/home/dmc/Desktop/kostas/direct-Behavior-prediction-from-miniscope-calcium-imaging-using-convolutional-neural-networks/data/tmaze_2021-10-26T14_29_27.csv", 
                   "/home/dmc/Desktop/kostas/direct-Behavior-prediction-from-miniscope-calcium-imaging-using-convolutional-neural-networks/data/tmaze_2021-10-27T16_50_53.csv"]
else:
    bonsai_paths = ["/home/dmc/Desktop/kostas/direct-Behavior-prediction-from-miniscope-calcium-imaging-using-convolutional-neural-networks/data/tmaze_2021-10-28T18_13_23.csv"]

    
    
    
if multiple_videos == True:

    if labels_type == 'old':


        behavior_paths = ["/home/dmc/Desktop/kostas/direct-Behavior-prediction-from-miniscope-calcium-imaging-using-convolutional-neural-networks/data/20211025_184906_animal3learnday8.h5", 
                       "/home/dmc/Desktop/kostas/direct-Behavior-prediction-from-miniscope-calcium-imaging-using-convolutional-neural-networks/data/20211026_142935_animal3learnday9.h5", 
                       "/home/dmc/Desktop/kostas/direct-Behavior-prediction-from-miniscope-calcium-imaging-using-convolutional-neural-networks/data/20211027_165052_animal3learnday10.h5"]

        df_new_annotations = align_files_old_labels(bonsai_paths, behavior_paths, num_of_videos)
    else:
        h5_path = "/home/dmc/Desktop/kostas/direct-Behavior-prediction-from-miniscope-calcium-imaging-using-convolutional-neural-networks/data/behavior_segmentation_arrowmaze.h5"
        df_new_annotations = align_files_new_labels(bonsai_paths, num_of_videos, h5_path)

        
else:
    if labels_type == 'old':
        behavior_paths = "/home/dmc/Desktop/kostas/direct-Behavior-prediction-from-miniscope-calcium-imaging-using-convolutional-neural-networks/data/20211028_181307_animal3learnday11.h5"
        df_new_annotations = align_files_old_labels(bonsai_paths, behavior_paths, num_of_videos)


    else:
        h5_path = "/home/dmc/Desktop/kostas/direct-Behavior-prediction-from-miniscope-calcium-imaging-using-convolutional-neural-networks/data/behavior_segmentation_arrowmaze.h5"
        df_new_annotations = align_files_new_labels(bonsai_paths, num_of_videos, h5_path)


In [ ]:
# behavior_paths = ["/home/dmc/Desktop/kostas/direct-Behavior-prediction-from-miniscope-calcium-imaging-using-convolutional-neural-networks/data/20211025_184906_animal3learnday8.h5", 
#                "/home/dmc/Desktop/kostas/direct-Behavior-prediction-from-miniscope-calcium-imaging-using-convolutional-neural-networks/data/20211026_142935_animal3learnday9.h5", 
#                "/home/dmc/Desktop/kostas/direct-Behavior-prediction-from-miniscope-calcium-imaging-using-convolutional-neural-networks/data/20211027_165052_animal3learnday10.h5"]

In [ ]:
df_new_annotations

In [ ]:
# df_new_annotations = align_files(bonsai_paths, behavior_paths, num_of_videos)
# df_new_annotations = align_files(bonsai_paths, num_of_videos, h5_path)

In [ ]:
# df_new_annotations = df_new_annotations.loc[:, 'state_id']    

In [ ]:
df_new_annotations = df_new_annotations.reset_index(drop=True)
# df_new_annotations = df_new_annotations.reset_index()

In [ ]:
df_new_annotations_unique = df_new_annotations.unique()

In [ ]:
df_new_annotations_unique

In [ ]:
save_dir = "/home/dmc/Desktop/kostas/direct-Behavior-prediction-from-miniscope-calcium-imaging-using-convolutional-neural-networks/src/output-svm/balance"

In [ ]:
if labels_type == 'old':
    class_counts, total_counts = check_class_imbalance_old(df_new_annotations, experiment_ID, save_dir)
else:
    class_counts, total_counts = check_class_imbalance_new(df_new_annotations, experiment_ID, save_dir)

In [ ]:
labels = df_new_annotations.values

In [ ]:
labels

In [ ]:
labels.shape

In [ ]:
type(labels)

In [ ]:
# data verification
# plt.plot(traces[50])

In [ ]:
type(traces[50])

In [ ]:
traces.shape

In [ ]:
num_classes = len(df_new_annotations_unique)

In [ ]:
num_classes

In [ ]:
# traces, labels, num_classes = model_preprocessing(traces, labels, df_new_annotations_unique)

In [ ]:
# labels

In [ ]:
labels = labels.astype(np.int32)

## Perform k-fold cross validation

In [ ]:
mySVC = sklearn.svm.LinearSVC()

In [ ]:
from sklearn.metrics import accuracy_score, log_loss, confusion_matrix
from sklearn.model_selection import cross_val_predict, KFold
from sklearn.svm import LinearSVC
 
mySVC.fit(traces.T, labels)
mySVC.predict(traces.T)

mySVC = LinearSVC()

kf = KFold(n_splits=5)

y_pred = cross_val_predict(mySVC, traces.T, labels, cv=kf)

accuracy = accuracy_score(labels, y_pred)
print("Average accuracy:", accuracy)

y_pred_prob = cross_val_predict(mySVC, traces.T, labels, cv=kf, method='decision_function')
log_loss_avg = log_loss(labels, y_pred_prob)
print("Average log loss:", log_loss_avg)

cm_avg = confusion_matrix(labels, y_pred)
print("Confusion matrix:")
print(cm_avg)

In [ ]:
# Shuffle the labels
shuffled_labels = np.random.permutation(labels)

# Use the shuffled labels for cross-validation
y_pred = cross_val_predict(mySVC, traces.T, shuffled_labels, cv=kf)

# Calculate accuracy using shuffled labels
accuracy = accuracy_score(shuffled_labels, y_pred)
print("Shuffled accuracy:", accuracy)